<a href="https://colab.research.google.com/github/Serena-G-LEE/25-2_Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 드라이브 마운트 & 기본 설정

Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


크롤링된 데이터 경로 지정(14개 대학/15년 논문)

In [ ]:
RAW_DIR = "/content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트"

전처리된 데이터 경로 지정

In [ ]:
OUT_DIR = "/content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트"

In [ ]:
import os
os.makedirs(OUT_DIR, exist_ok=True)

## 유틸함수: 전처리/깨짐판별/불용어/파일 처리

In [ ]:
# ===== 유틸/불용어/정규화 규칙 =====
import os, re, unicodedata
import pandas as pd
import numpy as np
from glob import glob
from html import unescape

# 프로젝트 루트는 OUT_DIR과 동일 경로로 사용
PROJ = OUT_DIR

# 불용어(초안) — 필요시 자유롭게 추가/제거하세요.
ko_stop = set("""
그리고 그러나 그래서 또한 또는 및 등이 등으로 등의 대한 대해서 대한의 를 을 은 는 이 가 에 의 와 과 도 에서 으로 로 에게 한 통해 각 그 그것 그것의 그중 그냥 등 등등
년 월 일 연구 결과 방법 토의 결론 자료 데이터 모델 기반 새로운 제안 논문 학술 학회 저널 이번 해당
""".split())

en_stop = set("""
a an the and or but if then else when while of for to in on at by from with about against between into through during
before after above below over under again further then once here there all any both each few more most other some such
no nor not only own same so than too very s t can will just don should now is are was were be been being do does did
doing have has had having i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she
her hers herself it its itself they them their theirs themselves what which who whom this that these those am because until
as up down out off over under why how
january february march april may june july august september october november december et al al vol no pp doi preprint
conference journal proceedings article study research results methods discussion conclusion data model models analysis
analyses using use used based novel new propose proposed large small significant significantly
""".split())

# HTML/TeX/마크업 잔여물도 노이즈 취급
extra_noise_tokens = {
    "scp","sub","sup","span","div","p","br","em","strong","ref","fig","tab","cite",
    "nbsp","amp","lt","gt","math","eq","section","table","figure",
    # 태그 조각
    "</scp>","</sub>","</sup>","</span>","</div>","</p>","</br>",
}
en_stop |= {t.lower() for t in extra_noise_tokens}

# 사용자 제보: 깨짐으로 자주 보이는 음절 블랙리스트
rare_hangul_blacklist = {"륛","뱓","릒"}

CTRL_CHARS = "".join(map(chr, list(range(0,32)) + [127]))
CTRL_RE = re.compile(f"[{re.escape(CTRL_CHARS)}]")
KEEP_RE = re.compile(r"[A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s\-\.,:;!?'\"()\[\]/%&+#@]")

def normalize_text(s: str) -> str:
    """
    1) HTML 엔티티 해제 + 유니코드 정규화(NFKC)
    2) 태그/엔티티/LaTeX/수식 등 마크업 제거
    3) 제어문자 제거, 사용자 블랙리스트 문자 제거
    4) ★ 허용 문자(영어/숫자/한글/일부 구두점)만 남김 → 한자/중국어 등은 전부 삭제
    5) 공백 압축
    """
    if not isinstance(s, str):
        return s
    s = unescape(s)
    s = unicodedata.normalize("NFKC", s)

    # 태그/엔티티/LaTeX/인라인 수식 제거
    s = re.sub(r"</?\s*[A-Za-z][^>]*>", " ", s)     # <tag ...> 또는 </tag>
    s = re.sub(r"&[A-Za-z]+;", " ", s)              # &nbsp; 등 엔티티
    s = re.sub(r"\\[A-Za-z]+", " ", s)              # \cite, \alpha ...
    s = re.sub(r"\$[^$]*\$", " ", s)                # $...$ 수식
    s = re.sub(r"\{[^{}]{0,80}\}", " ", s)          # 짧은 {...}

    # 제어문자/제로폭 제거
    s = CTRL_RE.sub(" ", s)
    s = s.replace("\u200b"," ").replace("\u200c"," ").replace("\u200d"," ")

    # 사용자 제보 블랙리스트 음절 제거
    for ch in rare_hangul_blacklist:
        s = s.replace(ch, " ")

    # === 핵심: 허용 문자만 남기고 나머지는 모두 공백으로 치환 ===
    # (즉, 한자/중국어/CJK 기호, 그 외 스크립트는 모두 제거됩니다)
    s = "".join(ch if ALLOW_RE.match(ch) else " " for ch in s)

    # 공백 압축
    s = re.sub(r"\s+", " ", s).strip()
    return s

def garbled_score(s):
    """깨짐 점수(높을수록 나쁨). 임계값 이상인 행은 삭제 대상."""
    if not isinstance(s, str) or not s:
        return 0.0
    # 깨짐·대체문자/박스문자
    bad = len(re.findall(r"\ufffd|�|\u25a1|\u25a0|\uFFFE|\uFFFF", s))
    # 동일문자 7연속 이상
    long_runs = len(re.findall(r"(.)\1{6,}", s))
    kept = len("".join(KEEP_RE.findall(s)))
    total = len(s)
    nonstd_prop = 1 - (kept / total if total else 1)
    # 블랙리스트 문자가 포함되면 가산
    penalty = 0.15 if any(ch in s for ch in rare_hangul_blacklist) else 0.0
    return bad/max(1,total) + long_runs*0.02 + nonstd_prop*0.5 + penalty

def remove_stopwords(s):
    """간단 토크나이즈 후 불용어/블랙리스트 토큰 제거."""
    if not isinstance(s, str):
        return s
    tokens = re.findall(r"[A-Za-z]+|[가-힣]+|\d+", s)
    cleaned = []
    for tok in tokens:
        low = tok.lower()
        if low in en_stop or tok in ko_stop or tok in rare_hangul_blacklist:
            continue
        cleaned.append(tok)
    return " ".join(cleaned)

def find_text_columns(df):
    keys = [
        "title","display_name","abstract","summary","name",
        "host_venue","journal","publisher","language",
        "author","authorships","institution","concept","keyword","topic","source"
    ]
    tcols = [c for c in df.columns if df[c].dtype==object and any(k in c.lower() for k in keys)]
    return tcols if tcols else (["title"] if "title" in df.columns else [])

def process_csv(in_path, out_dir,
                garbled_threshold=0.35,
                drop_when_clean_empty=True,
                dedupe_keys=("title","display_name","doi")):
    """CSV 하나 처리 → out_dir에 *.cleaned.csv 저장."""
    # 읽기(인코딩 이슈 시 치환)
    try:
        df = pd.read_csv(in_path)
    except Exception:
        df = pd.read_csv(in_path, encoding="utf-8", errors="replace")
    orig_len = len(df)

    text_cols = find_text_columns(df)
    for c in text_cols:
        df[c] = df[c].astype(str).map(normalize_text)

    # 깨짐 점수 → 임계값 이상 삭제
    score_col = next((c for c in ["title","display_name","abstract","summary","name"] if c in df.columns),
                     (text_cols[0] if text_cols else None))
    if score_col:
        df["_garbled_score"] = df[score_col].map(garbled_score)
        df = df[df["_garbled_score"] < garbled_threshold].copy()

    # 불용어 제거 버전 생성
    for c in text_cols:
        df[c+"_clean"] = df[c].map(remove_stopwords)

    # *_clean이 비면 삭제(정보 무의미)
    title_like = next((t for t in ["title","display_name","name"] if t in df.columns), None)
    if drop_when_clean_empty and title_like and (title_like+"_clean") in df.columns:
        df = df[df[title_like+"_clean"].astype(str).str.strip().astype(bool)].copy()

    # 중복 제거(제목/표제/DOI)
    subset = [k for k in dedupe_keys if k in df.columns]
    if subset:
        df = df.drop_duplicates(subset=subset)

    os.makedirs(out_dir, exist_ok=True)
    base = os.path.basename(in_path)
    name, ext = os.path.splitext(base)
    out_path = os.path.join(out_dir, f"{name}.cleaned.csv")
    df.to_csv(out_path, index=False)
    return f"{base}: {orig_len} → {len(df)} (삭제 {orig_len-len(df)}) → {out_path}"


## 일괄 처리 실행(모든 학교/특정 연도 파일 대상)

In [ ]:
from glob import glob
from tqdm import tqdm

# 처리할 연도 범위
YEARS = range(2011, 2026)  # 2011~2025

# 파라미터(원하시는 대로 조정)
GARBLED_THRESHOLD = 0.35       # 더 엄격: 0.25 / 더 느슨: 0.45
DROP_WHEN_CLEAN_EMPTY = True
DEDUPE_KEYS = ("title","display_name","doi")

all_logs = []
for y in YEARS:
    in_dir  = os.path.join(PROJ, f"openalex_top10000_{y}_with_authors")
    out_dir = os.path.join(PROJ, f"openalex_data_cleaning_{y}_with_authors")

    files = sorted(glob(os.path.join(in_dir, "*.csv")))
    print(f"\n=== {y}년: 입력 {len(files)}개 / 출력 폴더 → {out_dir}")
    if not files:
        all_logs.append(f"[경고] 입력 폴더 비어있음: {in_dir}")
        continue

    for fp in tqdm(files):
        try:
            log = process_csv(
                fp, out_dir,
                garbled_threshold=GARBLED_THRESHOLD,
                drop_when_clean_empty=DROP_WHEN_CLEAN_EMPTY,
                dedupe_keys=DEDUPE_KEYS
            )
        except Exception as e:
            log = f"[에러] {fp}: {e}"
        all_logs.append(log)

print("\n--- 처리 로그(일부) ---")
print("\n".join(all_logs[:80]))
print("\n완료.")



=== 2011년: 입력 25개 / 출력 폴더 → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors


100%|██████████| 25/25 [02:03<00:00,  4.95s/it]



=== 2012년: 입력 25개 / 출력 폴더 → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2012_with_authors


100%|██████████| 25/25 [02:09<00:00,  5.20s/it]



=== 2013년: 입력 25개 / 출력 폴더 → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2013_with_authors


...


=== 2024년: 입력 25개 / 출력 폴더 → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2024_with_authors


100%|██████████| 25/25 [03:12<00:00,  7.71s/it]



=== 2025년: 입력 25개 / 출력 폴더 → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2025_with_authors


100%|██████████| 25/25 [01:57<00:00,  4.72s/it]


--- 처리 로그(일부) ---
institutions_mapping.csv: 23 → 23 (삭제 0) → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors/institutions_mapping.cleaned.csv
works_top10000_2011_ALL.csv: 43164 → 43156 (삭제 8) → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors/works_top10000_2011_ALL.cleaned.csv
works_top10000_2011_Chonnam_National_University.csv: 1782 → 1782 (삭제 0) → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors/works_top10000_2011_Chonnam_National_University.cleaned.csv
works_top10000_2011_Chung-Ang_University.csv: 1490 → 1490 (삭제 0) → /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors/works_top10000_2011_Chung-Ang_University.cleaned.csv
works_top10000_2011_Chungbu